Imports

In [86]:
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import pprint
from gspread_dataframe import get_as_dataframe
import time
from datetime import date
import gspread
import datetime
import plotly.express as px


Connect to Google Sheets

In [87]:
service_account=gspread.service_account(filename="gambling-7-24-bf73ca0313d0.json")

In [88]:
sheet=service_account.open("Discipline")

In [89]:
worksheet=sheet.worksheet("Picks")

In [90]:
df = pd.DataFrame(worksheet.get_all_records())

Dataframe Calculations

###### Running Total

In [92]:
df['Win/Lose']=df["Win/Lose"].replace({'\$':''}, regex = True)
df['Win/Lose']=pd.to_numeric(df["Win/Lose"])
running_total=df['Win/Lose'].sum()

###### Week Total

In [93]:
t1 = datetime.datetime.now()
current_week=t1.strftime("%U")
df['Week']=df['Week'].astype(str)
weekly_sum=df[df.Week==current_week]['Win/Lose'].sum()

Visualizations

In [135]:
sport = px.histogram(df, 
            x="Week", 
            y="Win/Lose", 
            title=f"Historical Record"
            ,histfunc="sum"
            ,text_auto=True)

sport.update_layout(yaxis_title="Dollars ($)")
sport.update_xaxes(type='category')
sport.update_xaxes(categoryorder='total descending')

sport.show()

In [123]:
sport = px.histogram(df, 
            x="Sport", 
            y="Win/Lose", 
            color="Gender", 
            barmode="group",
            title=f"Historical Record by Sport/Gender"
            ,histfunc="sum"
            ,text_auto=True)

sport.update_layout(yaxis_title="Dollars ($)")
sport.update_xaxes(type='category')
sport.update_xaxes(categoryorder='total descending')
sport.update_layout(xaxis={'categoryorder':'total descending'}) # add only this line

sport.show()

In [113]:
week = px.histogram(df, 
            x="Week", 
            y="Win/Lose", 
            color="Sport", 
            barmode="group",
            title=f"Historical Record by Week/Sport"
            ,histfunc="sum"
            ,text_auto=True)

week.update_layout(yaxis_title="Dollars ($)")
week.update_xaxes(categoryorder='total descending')
week.update_yaxes(categoryorder='total descending')

week.show()

In [122]:
week = px.histogram(df, 
            x="Sport", 
            y="Win/Lose", 
            color="League", 
            barmode="group",
            title=f"Historical Record by Sport/League"
            ,histfunc="sum"
            ,text_auto=True)

week.update_layout(yaxis_title="Dollars ($)")
week.update_yaxes(categoryorder='total ascending')

week.show()